## All the import

In [1]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers.convolutional import AveragePooling2D,AveragePooling1D
from keras.callbacks import EarlyStopping
from keras.models import model_from_json
from keras.models import model_from_yaml
from keras.models import load_model


from sklearn.metrics import roc_curve, auc
import numpy as np
import RNA
from random import randint

import random
import gzip
from sklearn import metrics
from sklearn.metrics import roc_auc_score

from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse
import pandas as pd
import csv
from random import shuffle
#from keras.layers import Merge

from math import sqrt

Using TensorFlow backend.


## Load the data

In [2]:
def generate_rna(longueur):
    seq = ''
    for i in range(longueur):
        rand=randint(0,3)
        if rand == 0:
            seq = seq + 'a'
        elif rand == 1:
            seq = seq + 'g'
        elif rand == 2:
            seq = seq + 'c'
        else:
            seq = seq + 'u'
    return seq

In [3]:
s1=generate_rna(100)+'aacccccg'+generate_rna(25)
s2=generate_rna(75)+'ggggggguuuuuuuuuaaaaa'+generate_rna(55)
len1 = len(s1)
len2 = len(s2)
a = RNA.duplexfold(s1,s2)
struct1, struct2 = a.structure.split('&')
print(s1)
print(len1)
print(s2)
print(len2)
temp1 = '.'*(a.i-len(struct1))
temp2 = '.'*(len1-a.i)
struct1 = temp1+struct1+temp2
temp1 = '.'*(a.j-1)
temp2 = '.'*(len2-len(struct2)-a.j+1)
struct2=temp1+struct2+temp2
print()
print(struct1)
print(struct2)
print(len(struct1))
print(len(struct2))
a = RNA.subopt(s1+'&'+s2, '', 10)
struct1,struct2 = a.structure.split('&')
print()
print(struct1)
print(struct2)
print(len(struct1))
print(len(struct2))

gcaugaauuggcguucuacaaguuucuaagguuaauaagguuaaaaagccuaaccagggucucguggcuauggggcuaacagcccauuacaccuccugcgaacccccgugcgguaccugucgaaucauugagc
133
gauacgaaguagcucaccccaaauaagauucaugggcgugacuagcacuucaacgccggcaaguccggcugcaucggggggguuuuuuuuuaaaaaugcgcccuggguccgguugucgccagcugacccauccacggaauacagaagcgau
151

........(.((.((((....((((((..((.........................(((((..(((((.((.(((((....((.((((..............((((((((((((((.(((((...(.(((((.
................................................))))).).))))).))...)))))).))))))..............)))).))....))))).)).))))).....)))))..))..)))).)))))))).).
133
151

((........)).........((((((..((((....(((((....))))))))).((((.((((((.((..(((((...)))))..))..))....))))))))(((((.(((...((((............
)))).(((((.((((((((((...........)))).))))...)))))))...(((((.....)))))......(((.(.((((((....)))))).).))).((((.((((((....)))))))))))))))))).....))))))...
133
151


In [4]:
def get_predictions(structure, longueur, sense = 0):
    length = len(structure)
    prediction = np.zeros(length)
    index = 0
    nbparent = 0
    
    if(sense != 0):
        structure = structure[::-1]

    for char in structure[:85]:
        if char == '(' or char == ')':
            prediction[index] = 1
            index+=1
    
    for char in structure[85:85+longueur]:
        if char == '(' or char == ')':
            prediction[index] = 1
            nbparent+=1
        if(nbparent == 10):
            ones = np.ones(longueur)
            for i in range(longueur):
                prediction[i+85] = ones[i]
            break
        index+=1
    
    index = 85 + longueur
    for char in structure[85+longueur:]:
        if char == '(' or char == ')':
            prediction[index] = 1
        index+=1
    
    return prediction

In [5]:
def get_rna_subopt(s1, s2, longueur1, longueur2, direction = 1):
    print("RNA subopt")
    a = RNA.subopt(s1+'&'+s2, '', 10)
    structure = a.structure
    struct1,struct2 = structure.split('&')
    prediction1 = get_predictions(struct1, longueur1, sense=direction)
    prediction2 = get_predictions(struct2, longueur2, sense=0)
    '''
    print("Sequence 1 : ", s1)
    print("Structure 1 :", struct1)
    print("Sequence 2 : ", s2)
    print("Structure 2 :", struct2)
    print("Energy of the interaction : ", a.energy)
    '''
    return prediction1, prediction2

In [6]:
def get_rna_duplex(s1, s2, longueur1, longueur2, direction = 1):
    print("RNA duplex")
    a = RNA.duplexfold(s1,s2)
    struct1,struct2 = a.structure.split('&')
    i = int(a.i)
    j = int(a.j)
    len1 = len(s1)
    len2 = len(s2)
    temp1 = '.'*(i-len(struct1))
    temp2 = '.'*(len1-i)
    struct1 = temp1+struct1+temp2
    temp1 = '.'*(j-1)
    temp2 = '.'*(len2-len(struct2)-j+1)
    struct2=temp1+struct2+temp2
    prediction1 = get_predictions(struct1, longueur1, sense=direction)
    prediction2 = get_predictions(struct2, longueur2, sense=0)
    '''
    print("Sequence 1 : ", s1)
    print("Structure 1 :", struct1)
    print("Sequence 2 : ", s2)
    print("Structure 2 :", struct2)
    print("Position of last parenthese in the first sequence : ", a.i)
    print("Position of the first parenthese in the second sequence : ", a.j)
    '''
    return prediction1, prediction2

In [7]:
def calculate_performance(test_num, pred_y, labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    if (tp+fp) == 0:
        precision = 0
    else:
        precision = float(tp)/(tp+ fp)
    if (tp+fn) == 0:
        sensitivity = 0
    else:
        sensitivity = float(tp)/ (tp+fn)
    if (tn+fp) == 0:
        specificity = 0  
    else:
        specificity = float(tn)/(tn + fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    print("Accuracy : ", acc)
    print("Sensitivity : ", sensitivity)
    print("Specificity : ", specificity)
    print("MCC : ", MCC)
    return acc, precision, sensitivity, specificity, MCC

In [8]:
def load_benchmark():
    print("Loading benchmark database")
    datatest = pd.read_csv("genomes/benchmark_interactionsv2.csv", sep = "\t", header=None)
    datatest = np.array(datatest)
    sequence1=[]
    sequence2=[]
    longueur1=[]
    longueur2=[]
    l = len(datatest)
    for i in range(l):
        data = datatest[i][0].split(' ')
        data[0] = data[0].split('[')
        data[0][1] = data[0][1].split('\n')
        data[1] = data[1].split('\n')
        seq1=data[0][1][0]
        seq2=data[1][0]
        len1=int(data[2])
        len2=int(data[3])
        sequence1.append(seq1)
        sequence2.append(seq2)
        longueur1.append(len1)
        longueur2.append(len2)
    print("Benchmark database loaded")
    return sequence1, sequence2, longueur1, longueur2    

In [10]:
def get_score_RNA():
    benchmark = load_benchmark()
    sequence1 = benchmark[0]
    sequence2 = benchmark[1]
    longueur1 = benchmark[2]
    longueur2 = benchmark[3]
    score_subopt1 = []
    score_duplex1 = []
    score_subopt2 = []
    score_duplex2 = []
    length = len(sequence1)
    print("Executing")
    for i in range(length):
        print("Index : ", i)
        seq1 = sequence1[i]
        seq2 = sequence2[i]
        len1 = longueur1[i]
        len2 = longueur2[i]
        label1 = np.zeros(len(seq1))
        label2 = np.zeros(len(seq2))
        ones1 = np.ones(len1)
        ones2 = np.ones(len2)
        label1[85:85+len1] = ones1[:]
        label2[85:85+len2] = ones2[:]
        
        prediction1, prediction2 = get_rna_subopt(seq1,seq2,len1,len2)
        score_subopt1.append(calculate_performance(len(seq1), prediction1, label1))
        score_subopt2.append(calculate_performance(len(seq2), prediction2, label2))
        prediction3, prediction4 = get_rna_duplex(seq1,seq2,len1,len2)
        score_duplex1.append(calculate_performance(len(seq1), prediction3, label1))
        score_duplex2.append(calculate_performance(len(seq2), prediction4, label2))
    return score_subopt1, score_subopt2, score_duplex1, score_duplex2

In [11]:
get_score_RNA()

Loading benchmark database
Benchmark database loaded
Executing
Index :  0
RNA subopt
Accuracy :  0.6767676767676768
Sensitivity :  0.0
Specificity :  0.7790697674418605
MCC :  -0.18947602167740824
Accuracy :  0.49473684210526314
Sensitivity :  0.0
Specificity :  0.5465116279069767
MCC :  -0.2699667753644223
RNA duplex
Accuracy :  0.6313131313131313
Sensitivity :  0.0
Specificity :  0.7267441860465116
MCC :  -0.21691166393777972
Accuracy :  0.6578947368421053
Sensitivity :  0.0
Specificity :  0.7267441860465116
MCC :  -0.18546114612343503
Index :  1
RNA subopt
Accuracy :  0.4900990099009901
Sensitivity :  1.0
Specificity :  0.4011627906976744
MCC :  0.30081198035615403
Accuracy :  0.2857142857142857
Sensitivity :  0.0
Specificity :  0.3372093023255814
MCC :  -0.48047859599823795
RNA duplex
Accuracy :  0.5742574257425742
Sensitivity :  1.0
Specificity :  0.5
MCC :  0.3595974761140381
Accuracy :  0.3694581280788177
Sensitivity :  0.0
Specificity :  0.436046511627907
MCC :  -0.406115493141

Index :  22
RNA subopt
Accuracy :  0.336734693877551
Sensitivity :  1.0
Specificity :  0.2441860465116279
MCC :  0.19507682662870654
Accuracy :  0.46798029556650245
Sensitivity :  1.0
Specificity :  0.37209302325581395
MCC :  0.28807078118184964
RNA duplex
Accuracy :  0.4387755102040816
Sensitivity :  1.0
Specificity :  0.36046511627906974
MCC :  0.2540884026249331
Accuracy :  0.4630541871921182
Sensitivity :  1.0
Specificity :  0.36627906976744184
MCC :  0.2847887834216627
Index :  23
RNA subopt
Accuracy :  0.4072164948453608
Sensitivity :  0.0
Specificity :  0.45930232558139533
MCC :  -0.34318468943336944
Accuracy :  0.2946859903381642
Sensitivity :  0.0
Specificity :  0.3546511627906977
MCC :  -0.48506023361285566
RNA duplex
Accuracy :  0.5773195876288659
Sensitivity :  1.0
Specificity :  0.5232558139534884
MCC :  0.3326991582117039
Accuracy :  0.42995169082125606
Sensitivity :  0.0
Specificity :  0.5174418604651163
MCC :  -0.36906087097065765
Index :  24
RNA subopt
Accuracy :  0.35

Accuracy :  0.5526315789473685
Sensitivity :  1.0
Specificity :  0.5058139534883721
MCC :  0.29731241394096686
Accuracy :  0.5368421052631579
Sensitivity :  1.0
Specificity :  0.4883720930232558
MCC :  0.2879776208944448
Index :  40
RNA subopt
Accuracy :  0.3958333333333333
Sensitivity :  1.0
Specificity :  0.32558139534883723
MCC :  0.21881411509101575
Accuracy :  0.2356020942408377
Sensitivity :  0.0
Specificity :  0.2616279069767442
MCC :  -0.46819249792823364
RNA duplex
Accuracy :  0.4739583333333333
Sensitivity :  1.0
Specificity :  0.4127906976744186
MCC :  0.2612084749471223
Accuracy :  0.34554973821989526
Sensitivity :  0.0
Specificity :  0.38372093023255816
MCC :  -0.37115562497154264
Index :  41
RNA subopt
Accuracy :  0.3553299492385787
Sensitivity :  1.0
Specificity :  0.2616279069767442
MCC :  0.20743892053506732
Accuracy :  0.4895833333333333
Sensitivity :  1.0
Specificity :  0.43023255813953487
MCC :  0.2700384649749418
RNA duplex
Accuracy :  0.48223350253807107
Sensitivi

Accuracy :  0.359375
Sensitivity :  1.0
Specificity :  0.28488372093023256
MCC :  0.19960931103828644
Accuracy :  0.37435897435897436
Sensitivity :  1.0
Specificity :  0.29069767441860467
MCC :  0.21473394141922195
Index :  64
RNA subopt
Accuracy :  0.4030612244897959
Sensitivity :  1.0
Specificity :  0.31976744186046513
MCC :  0.23329916575548837
Accuracy :  0.39896373056994816
Sensitivity :  1.0
Specificity :  0.32558139534883723
MCC :  0.22339793473363267
RNA duplex
Accuracy :  0.45408163265306123
Sensitivity :  1.0
Specificity :  0.37790697674418605
MCC :  0.26312519067622636
Accuracy :  0.46632124352331605
Sensitivity :  1.0
Specificity :  0.4011627906976744
MCC :  0.2606508386998376
Index :  65
RNA subopt
Accuracy :  0.38461538461538464
Sensitivity :  1.0
Specificity :  0.3023255813953488
MCC :  0.22051256837613434
Accuracy :  0.3711340206185567
Sensitivity :  1.0
Specificity :  0.29069767441860467
MCC :  0.21074189119119713
RNA duplex
Accuracy :  0.4512820512820513
Sensitivity :

Sensitivity :  1.0
Specificity :  0.5290697674418605
MCC :  0.3913461061334349
Accuracy :  0.3385416666666667
Sensitivity :  1.0
Specificity :  0.2616279069767442
MCC :  0.1886680513554908
RNA duplex
Accuracy :  0.6439024390243903
Sensitivity :  1.0
Specificity :  0.5755813953488372
MCC :  0.4233089113177906
Accuracy :  0.46875
Sensitivity :  0.0
Specificity :  0.5232558139534884
MCC :  -0.2944159661080998
Index :  90
RNA subopt
Accuracy :  0.4947916666666667
Sensitivity :  1.0
Specificity :  0.436046511627907
MCC :  0.27301623621231697
Accuracy :  0.44502617801047123
Sensitivity :  1.0
Specificity :  0.38372093023255816
MCC :  0.24150689720036744
RNA duplex
Accuracy :  0.53125
Sensitivity :  1.0
Specificity :  0.47674418604651164
MCC :  0.2944159661080998
Accuracy :  0.4031413612565445
Sensitivity :  0.0
Specificity :  0.4476744186046512
MCC :  -0.33062734004388705
Index :  91
RNA subopt
Accuracy :  0.5048543689320388
Sensitivity :  1.0
Specificity :  0.4069767441860465
MCC :  0.31897

...

Specificity :  0.313953488372093
MCC :  0.21778847991634692
Index :  2355
RNA subopt
Accuracy :  0.47643979057591623
Sensitivity :  0.0
Specificity :  0.5290697674418605
MCC :  -0.28520605658819337
Accuracy :  0.4052631578947368
Sensitivity :  1.0
Specificity :  0.3430232558139535
MCC :  0.2171012815432438
RNA duplex
Accuracy :  0.5497382198952879
Sensitivity :  1.0
Specificity :  0.5
MCC :  0.3007926037591192
Accuracy :  0.4368421052631579
Sensitivity :  0.0
Specificity :  0.48255813953488375
MCC :  -0.3036731158186559
Index :  2356
RNA subopt
Accuracy :  0.4230769230769231
Sensitivity :  1.0
Specificity :  0.3023255813953488
MCC :  0.26413527189768715
Accuracy :  0.31313131313131315
Sensitivity :  1.0
Specificity :  0.20930232558139536
MCC :  0.18328047158971922
RNA duplex
Accuracy :  0.4567307692307692
Sensitivity :  1.0
Specificity :  0.3430232558139535
MCC :  0.2878855711491191
Accuracy :  0.398989898989899
Sensitivity :  1.0
Specificity :  0.3081395348837209
MCC :  0.235058729374

Specificity :  0.2558139534883721
MCC :  0.18592862386976444
RNA duplex
Accuracy :  0.583756345177665
Sensitivity :  1.0
Specificity :  0.5232558139534884
MCC :  0.34965129957937235
Accuracy :  0.5520833333333334
Sensitivity :  1.0
Specificity :  0.5
MCC :  0.30714755841697555
Index :  2373
RNA subopt
Accuracy :  0.34554973821989526
Sensitivity :  1.0
Specificity :  0.27325581395348836
MCC :  0.18988045334711914
Accuracy :  0.31794871794871793
Sensitivity :  1.0
Specificity :  0.22674418604651161
MCC :  0.18283936271530685
RNA duplex
Accuracy :  0.4816753926701571
Sensitivity :  1.0
Specificity :  0.42441860465116277
MCC :  0.2614165046138236
Accuracy :  0.4666666666666667
Sensitivity :  1.0
Specificity :  0.3953488372093023
MCC :  0.267579162704228
Index :  2374
RNA subopt
Accuracy :  0.4329896907216495
Sensitivity :  1.0
Specificity :  0.36046511627906974
MCC :  0.2451071589730601
Accuracy :  0.45
Sensitivity :  1.0
Specificity :  0.36046511627906974
MCC :  0.2704401036825183
RNA dup

MCC :  0.22990024493585143
RNA duplex
Accuracy :  0.43455497382198954
Sensitivity :  1.0
Specificity :  0.37209302325581395
MCC :  0.2359395265150154
Accuracy :  0.328125
Sensitivity :  0.0
Specificity :  0.36627906976744184
MCC :  -0.3907733852378575
Index :  2397
RNA subopt
Accuracy :  0.37894736842105264
Sensitivity :  1.0
Specificity :  0.313953488372093
MCC :  0.2038447143750835
Accuracy :  0.5794871794871795
Sensitivity :  1.0
Specificity :  0.5232558139534884
MCC :  0.3385527140628702
RNA duplex
Accuracy :  0.5684210526315789
Sensitivity :  1.0
Specificity :  0.5232558139534884
MCC :  0.3068974527617131
Accuracy :  0.5948717948717949
Sensitivity :  1.0
Specificity :  0.5406976744186046
MCC :  0.34917334466394057
Index :  2398
RNA subopt
Accuracy :  0.5368421052631579
Sensitivity :  1.0
Specificity :  0.4883720930232558
MCC :  0.2879776208944448
Accuracy :  0.465
Sensitivity :  1.0
Specificity :  0.37790697674418605
MCC :  0.27996554482236385
RNA duplex
Accuracy :  0.415789473684

MCC :  0.18451955816738946
Accuracy :  0.48292682926829267
Sensitivity :  1.0
Specificity :  0.38372093023255816
MCC :  0.30182646280989667
RNA duplex
Accuracy :  0.43455497382198954
Sensitivity :  1.0
Specificity :  0.37209302325581395
MCC :  0.2359395265150154
Accuracy :  0.5024390243902439
Sensitivity :  1.0
Specificity :  0.4069767441860465
MCC :  0.3154095815513012
Index :  2425
RNA subopt
Accuracy :  0.3582089552238806
Sensitivity :  1.0
Specificity :  0.25
MCC :  0.21421035444013226
Accuracy :  0.4484536082474227
Sensitivity :  1.0
Specificity :  0.37790697674418605
MCC :  0.2538685719970979
RNA duplex
Accuracy :  0.5323383084577115
Sensitivity :  1.0
Specificity :  0.45348837209302323
MCC :  0.3269862729360347
Accuracy :  0.5051546391752577
Sensitivity :  1.0
Specificity :  0.4418604651162791
MCC :  0.2870204952310829
Index :  2426
RNA subopt
Accuracy :  0.40932642487046633
Sensitivity :  1.0
Specificity :  0.3372093023255814
MCC :  0.22903008614973952
Accuracy :  0.39175257731

([(0.6767676767676768, 0.0, 0.0, 0.7790697674418605, -0.18947602167740824),
  (0.4900990099009901,
   0.22556390977443608,
   1.0,
   0.4011627906976744,
   0.30081198035615403),
  (0.4175257731958763,
   0.16296296296296298,
   1.0,
   0.3430232558139535,
   0.23643199050180216),
  (0.6701570680628273,
   0.23170731707317074,
   1.0,
   0.6337209302325582,
   0.3831941759960608),
  (0.4393939393939394,
   0.1897810218978102,
   1.0,
   0.3546511627906977,
   0.2594341151268762),
  (0.41450777202072536,
   0.15671641791044777,
   1.0,
   0.3430232558139535,
   0.23185636914077207),
  (0.3768844221105528,
   0.17880794701986755,
   1.0,
   0.27906976744186046,
   0.22338283772839612),
  (0.48058252427184467,
   0.24113475177304963,
   1.0,
   0.37790697674418605,
   0.3018716698044269),
  (0.38860103626943004,
   0.1510791366906475,
   1.0,
   0.313953488372093,
   0.21778847991634692),
  (0.2923076923076923, 0.0, 0.0, 0.3313953488372093, -0.4384333897062384),
  (0.4031413612565445, 0.0

In [ ]:
def plot_performance(tab):
    length = len(tab)
    

In [3]:
def traduire(seq):
    nucleotides = ''
    for lettre in seq:
            if lettre=='A' or lettre=='a':
                nucleotides=nucleotides+'a'
            elif lettre=='C' or lettre=='c':
                nucleotides=nucleotides+'c'
            elif lettre=='G' or lettre=='g':
                nucleotides=nucleotides+'g'
            else:
                nucleotides=nucleotides+'u'
    return nucleotides

In [3]:
def convert_sequence(sequence):
    matrice = np.zeros((36,4))
    for i in range(len(sequence)):
            if sequence[i]=='a':
                matrice[i,0] = 1
            elif sequence[i]=='u':
                matrice[i,1] = 1
            elif sequence[i]=='g':
                matrice[i,2] = 1
            elif sequence[i]=='c':
                matrice[i,3] = 1
    return matrice

In [4]:
def convert_sequence_fast(sequence):
    matrice = np.zeros((len(sequence),4))
    for i in range(len(sequence)):
            if sequence[i]=='a':
                matrice[i,0] = 1
            elif sequence[i]=='u':
                matrice[i,1] = 1
            elif sequence[i]=='g':
                matrice[i,2] = 1
            elif sequence[i]=='c':
                matrice[i,3] = 1
    return matrice

In [5]:
def convert_matrice3D(seq1,seq2):
    matrice = np.zeros((36,36,1))
    for j in range(len(seq1)):
        for k in range(len(seq2)):
            if (seq1[j][0]==1 and seq2[k][1]==1) or (seq1[j][1]==1 and seq2[k][0]==1):
                matrice[j,k,0] = 1
            elif (seq1[j][2]==1 and seq2[k][3]==1) or (seq1[j][3]==1 and seq2[k][2]==1):
                matrice[j,k,0] = 1
            elif (seq1[j][2]==1 and seq2[k][1]==1) or (seq1[j][1]==1 and seq2[k][2]==1):
                matrice[j,k,0] = 1
    return matrice

In [6]:
def convert_matrice3D_fast(seq1,seq2):
    matrice = np.zeros((len(seq1),len(seq2),1))
    for j in range(len(seq1)):
        for k in range(len(seq2)):
            if (seq1[j][0]==1 and seq2[k][1]==1) or (seq1[j][1]==1 and seq2[k][0]==1):
                matrice[j,k,0] = 1
            elif (seq1[j][2]==1 and seq2[k][3]==1) or (seq1[j][3]==1 and seq2[k][2]==1):
                matrice[j,k,0] = 1
            elif (seq1[j][2]==1 and seq2[k][1]==1) or (seq1[j][1]==1 and seq2[k][2]==1):
                matrice[j,k,0] = 1
    return matrice

In [7]:
def get_product_proba(proba1,proba2):
    product = np.zeros((36,36,1))
    for j in range(len(proba1)):
            for k in range(len(proba2)):
                product[j][k] = proba1[j] * proba2[k]
    return product

In [8]:
def get_sum_proba(proba1,proba2):
    somme = np.zeros((36,36,1))
    for j in range(len(proba1)):
            for k in range(len(proba2)):
                somme[j][k] = proba1[j] + proba2[k]
    return somme

In [9]:
def get_sum_proba_fast(proba1,proba2):
    somme = np.zeros((len(proba1),len(proba2),1))
    for j in range(len(proba1)):
            for k in range(len(proba2)):
                somme[j][k] = proba1[j] + proba2[k]
    return somme

In [10]:
def reverse(s):
    string = ''
    for i in range(len(s)):
        string = s[i] + string
    return string

In [11]:
def load_datatest_p_n(w = 36):
    datatest = pd.read_csv("genomes/nar-03123-met-g-2015-File010.csv", sep = ",", header=None)
    datatest = np.array(datatest)
    l = len(datatest)
    seq1 = np.zeros((200,36,4))
    seq2 = np.zeros((200,36,4))
    #seq3 = np.zeros((200,36,4))
    #seq4 = np.zeros((200,36,4))
    matrice3D = np.zeros((200,36,36,1))
    product = np.zeros((200,36,36,1))
    index = 0
    for i in range(1,l):
        #print("i : ",i)
        begin1 = int(datatest[i,9])-1
        end1 = int(datatest[i,10])
        size1 = end1-begin1
        if(size1 <= 36):

            begin2 = int(datatest[i,11])-1
            end2 = int(datatest[i,12])
            size2 = end2-begin2

            if(size2 <= 36):
                #print("taille 1 : ", size1)
                #print("taille 2 : ", size2)
                #print("index : ",index)

                sequence = traduire(datatest[i,23][begin1:begin1+w])
                z1 = len(sequence)
                print("sequence : ",sequence, " de longueur : ", z1)
                proba1 = np.array(RNA.pfl_fold_up(sequence,1,z1,z1))[1:,1]
                #proba1 = np.ones(w)*0.5
                seq1[index] = convert_sequence(sequence)

                sequence = traduire(datatest[i,24][begin2:begin2+w])
                z2 = len(sequence)
                print("sequence : ",sequence, " de longueur : ", z2)
                proba2 = np.array(RNA.pfl_fold_up(sequence, 1, z2, z2))[1:,1]
                #proba2 = np.ones(w)*0.5
                seq2[index] = convert_sequence(sequence)

                matrice3D[index] = convert_matrice3D(seq1[index],seq2[index])*0.8
                product[index] = get_sum_proba(proba1,proba2)                

                index+=1
    nb_pos = index


    for i in range(1,l):    
        #print("i : ",i)
        begin1 = int(datatest[i,9])-1
        end1 = int(datatest[i,10])
        #milieu1 = int((begin1+end1)/2)
        size1 = end1-begin1
        if(size1 <= 36):

            begin2 = int(datatest[i,11])-1
            end2 = int(datatest[i,12])
            size2 = end2-begin2

            if(size2 <= 36):

                #print("taille 1 : ", size1)
                #print("taille 2 : ", size2)
                #print("index : ",index)

                longueur1 = int(datatest[i,7])

                if (longueur1 - end1 < w):
                    if (begin1-w>=0):
                        sequence = traduire(datatest[i,23][begin1-w:begin1])
                    else: 
                        sequence = traduire(datatest[i,23][:w])
                else :
                    sequence = traduire(datatest[i,23][end1:end1+w])

                z1 = len(sequence)
                print("sequence : ",sequence, " de longueur : ", z1)
                #print(sequence)
                proba1 = np.array(RNA.pfl_fold_up(sequence,1,z1,z1))[1:,1]
                #proba1 = np.ones(w)*0.5
                seq1[index] = convert_sequence(sequence)

                sequence = traduire(datatest[i,24][end2+1:end2+w+1])
                z2 = len(sequence)
                print("sequence : ",sequence, " de longueur : ", z2)
                #print(sequence)
                proba2 = np.array(RNA.pfl_fold_up(sequence, 1, z2, z2))[1:,1]
                #proba2 = np.ones(w)*0.5
                seq2[index] = convert_sequence(sequence)

                matrice3D[index] = convert_matrice3D(seq1[index],seq2[index])*0.8
                product[index] = get_sum_proba(proba1,proba2)                

                index+=1

    l = len(seq1)
    revseq1 = np.zeros((l,36,4))
    revseq2 = np.zeros((l,36,4))

    for i in range(l):
        for j in range(36):
            revseq1[i][36-j-1] = seq1[i][j]
            revseq2[i][36-j-1] = seq2[i][j]

    test = []
    test.append(matrice3D[:index])
    test.append(product[:index])
    test.append(seq2[:index])
    test.append(revseq2[:index])
    test.append(seq1[:index])
    test.append(revseq1[:index])
    
    return test, nb_pos

In [12]:
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[2:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

## Calculate the performance

In [13]:
def calculate_performance(test_num, pred_y, labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    print(tp+tn)
    if (tp+fp) == 0:
        precision = 0
    else:
        precision = float(tp)/(tp+ fp)
    if (tp+fn) == 0:
        sensitivity = 0
    else:
        sensitivity = float(tp)/ (tp+fn)
    if (tn+fp) == 0:
        specificity = 0  
    else:
        specificity = float(tn)/(tn + fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC

In [14]:
def perfs_test(model,test,true):
    print('predicting')
            
    predictions = model.predict(test)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true)
    perfs = calculate_performance(len(predictions), predictions, true)
    
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])

In [17]:
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[2:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

def load_data(nb_train = 165000,nb_val = 6000,nb_test = 5000):
    # load your data using this function
    verif = []
    total = nb_train+nb_val+nb_test
    verif = pd.read_csv("genomes/negatifs_m-m-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            

    neg1 = verif

    verif=[]

    verif = pd.read_csv("genomes/negatifs_divers-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg2 = verif
    verif=[]
    verif = pd.read_csv("genomes/negatifs_mouse_divers-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg3 = verif
    verif=[]

    verif = pd.read_csv("genomes/positifs_m-m-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos1 = verif
    l = len(pos1)
    verif=[]

    verif = pd.read_csv("genomes/positifs_divers-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos2 = verif
    l=l+len(pos2)
    verif=[]

    verif = pd.read_csv("genomes/positifs_mouse_divers-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos3 = verif
    l=l+len(pos3)
    verif=[]

    bdd = np.concatenate((pos1,pos2,pos3,neg1,neg2,neg3))

    pos1 = []
    neg1 = []
    pos2 = []
    neg2 = []

    labels = np.zeros((len(bdd),1))

    bdd = np.concatenate((bdd,labels),axis=1)
    for i in range(l):
        bdd[i,4]=1
    labels=[]
    
    # shuffle total

    indices = np.arange(len(bdd))
    shuffle(indices)
    bdd = bdd[indices][:total]
    indices=[]

    # 
    l = len(bdd)
    matrice = np.zeros((l,36,36,1)) #4
    matrice1 = np.zeros((l,36,36,1))
    matrice2 = np.zeros((l,36,4))
    matrice3 = np.zeros((l,36,4))
    matrice4 = np.zeros((l,36,4))
    matrice5 = np.zeros((l,36,4))

    for i in range(l):
        seq1 = bdd[i,0]
        seq2 = bdd[i,1]
        prob1 = np.array(clean(bdd[i,2]))
        prob2 = np.array(clean(bdd[i,3]))
        for j in range(len(seq1)):
            for k in range(len(seq2)):
                if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                matrice1[i,j,k,0] = prob1[j]+prob2[k]
        for j in range(len(seq1)):
            if seq1[j]=='a':
                matrice2[i,j,0] = 1
            elif seq1[j]=='u':
                matrice2[i,j,1] = 1
            elif seq1[j]=='g':
                matrice2[i,j,2] = 1
            elif seq1[j]=='c':
                matrice2[i,j,3] = 1
        for j in range(len(seq2)):
            if seq2[j]=='a':
                matrice3[i,j,0] = 1
            elif seq2[j]=='u':
                matrice3[i,j,1] = 1
            elif seq2[j]=='g':
                matrice3[i,j,2] = 1
            elif seq2[j]=='c':
                matrice3[i,j,3] = 1

    for i in range(36):
        matrice4[:,36-i-1,:] = matrice2[:,i,:]
    for i in range(36):
        matrice5[:,36-i-1,:] = matrice3[:,i,:]
    
    training = []
    training.append(matrice[:nb_train])
    training.append(matrice1[:nb_train])
    training.append(matrice2[:nb_train])
    training.append(matrice4[:nb_train])
    training.append(matrice3[:nb_train])
    training.append(matrice5[:nb_train])    
    
    validation = []
    validation.append(matrice[nb_train:nb_train+nb_val])
    validation.append(matrice1[nb_train:nb_train+nb_val])
    validation.append(matrice2[nb_train:nb_train+nb_val])
    validation.append(matrice4[nb_train:nb_train+nb_val])
    validation.append(matrice3[nb_train:nb_train+nb_val])
    validation.append(matrice5[nb_train:nb_train+nb_val])

    testing = []
    testing.append(matrice[-nb_test:])
    testing.append(matrice1[-nb_test:])
    testing.append(matrice2[-nb_test:])
    testing.append(matrice4[-nb_test:])
    testing.append(matrice3[-nb_test:])
    testing.append(matrice5[-nb_test:])

    labels = bdd[:,4]
    bdd = []
    y = labels[:nb_train]
    y = keras.utils.np_utils.to_categorical(y,2)
    val_y = labels[nb_train:nb_train+nb_val]
    val_y = keras.utils.np_utils.to_categorical(val_y,2)
    true_y = labels[-nb_test:]
    argtest=[]
    np.sum(y[:,1])
    return training, y, validation, val_y, testing, true_y

In [20]:
def seq2augc():
    return

## Create The Model

### Data

In [18]:
Data = load_data(20000,0,0)

26023
24229
37991
26023
24229
37993


### Récupération du modèle

In [17]:
# paramètres 
#true_y = Data[5]
#training = Data[0]
#y = Data[1]
batch_size=48
epochs=25
verbose1 = 1
verbose2 = 1
#validation = Data[2]
#val_y = Data[3]
#testing = Data[4]
matrixsize11 = 6
nbfilter11 = 24
matrixsize12 = 7
nbfilter12 = 4
matrixsize21 = 6
nbfilter21 = 24
matrixsize22 = 7
nbfilter22 = 4
nbfilter1 = 64
kernelsize = 7
nbfilters2 = 64
kernel_size2 = 7
Dense1 = 128
Dense2 = 512
Dense3 = 512
Dense4 = 128
Dense5 = 0

from keras.layers import merge
from keras import Model
k = matrixsize11
# init_weights
I = np.eye(k)
M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
I2 = np.zeros((k,k))
M2 = np.zeros((k,k))
for j in range(k):
    I2[:,j] = I[:,k-j-1]
    M2[:,j] = M[:,k-j-1]        
W = np.zeros((k,k,1,nbfilter11))
W[:,:,0,0] = I
W[:,:,0,1] = I2
W[:,:,0,2] = M
W[:,:,0,3] = M2
for j in range(4,nbfilter11):
    W[:,:,0,j] = np.random.randn(k,k)*0.2


k2 = matrixsize12
I = np.eye(k2)
M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
I2=np.zeros((k2,k2))
M2=np.zeros((k2,k2))
for j in range(k2):
    I2[:,j] = I[:,k2-j-1]
    M2[:,j] = M[:,k2-j-1]   

Z = np.zeros((k2,k2,nbfilter11,nbfilter12))

for u in range(nbfilter12):
    Z[:,:,u,0] = I
    Z[:,:,u,1] = I2
    Z[:,:,u,2] = M
    Z[:,:,u,3] = M2
    for p in range(4,nbfilter12):
        Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

c2d1_input = keras.Input(shape=(36,36,1))
cnn2d1 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d1_input)
cnn2d1 = AveragePooling2D(pool_size=(3,3))(cnn2d1)
#cnn2d1 = MaxPooling2D(pool_size=(3,3))(cnn2d1)
cnn2d1 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d1)
cnn2d1 = Dropout(0.2)(cnn2d1)
cnn2d1 = Flatten()(cnn2d1)

c2d2_input = keras.Input(shape=(36,36,1))
cnn2d2 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d2_input)
cnn2d2 = AveragePooling2D(pool_size=(3,3))(cnn2d2)
#cnn2d2 = MaxPooling2D(pool_size=(3,3))(cnn2d2)
cnn2d2 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d2)
cnn2d2 = Dropout(0.2)(cnn2d2)
cnn2d2 = Flatten()(cnn2d2)

c1d1_input = keras.Input(shape=(36,4))
cnn1d1 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d1_input)
cnn1d1 = Activation('relu')(cnn1d1)
cnn1d1 = MaxPooling1D(pool_size=8)(cnn1d1)
cnn1d1 = Dropout(0.2)(cnn1d1)
cnn1d1 = Flatten()(cnn1d1)

c1d2_input = keras.Input(shape=(36,4))
cnn1d2 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d2_input)
cnn1d2 = Activation('relu')(cnn1d2)
cnn1d2 = MaxPooling1D(pool_size=8)(cnn1d2)
cnn1d2 = Dropout(0.2)(cnn1d2)
cnn1d2 = Flatten()(cnn1d2)

c1d3_input = keras.Input(shape=(36,4))
cnn1d3 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d3_input)
cnn1d3 = Activation('relu')(cnn1d3)
cnn1d3 = MaxPooling1D(pool_size=8)(cnn1d3)
cnn1d3 = Dropout(0.2)(cnn1d3)
cnn1d3 = Flatten()(cnn1d3)

c1d4_input = keras.Input(shape=(36,4))
cnn1d4 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d4_input)
cnn1d4 = Activation('relu')(cnn1d4)
cnn1d4 = MaxPooling1D(pool_size=8)(cnn1d4)
cnn1d4 = Dropout(0.2)(cnn1d4)
cnn1d4 = Flatten()(cnn1d4)

model21 = keras.layers.concatenate([cnn1d1,cnn1d2])
model21 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model21)
model21 = BatchNormalization()(model21)
model21 = Activation('relu')(model21)
model21 = Dropout(0.3)(model21)

model22 = keras.layers.concatenate([cnn1d3,cnn1d4])
model22 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model22)
model22 = BatchNormalization()(model22)
model22 = Activation('relu')(model22)
model22 = Dropout(0.3)(model22)

model2 = keras.layers.concatenate([model21,model22])
model2 = Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model2)
model2 = BatchNormalization()(model2)
model2 = Activation('relu')(model2)
model2 = Dropout(0.3)(model2)

model1 = keras.layers.concatenate([cnn2d1,cnn2d2])
model1 = Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model1)
model1 = Dropout(0.1)(model1)
model1 = BatchNormalization()(model1)
model1 = Activation('relu')(model1)

model = keras.layers.concatenate([model1,model2])
model = Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.2)(model)
model = Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.2)(model)

if Dense5>0:
    model = Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Dropout(0.2)(model)

model = Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
interaction_output = Activation('softmax')(model)

interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])

In [18]:
interaction.load_weights('model_finaldeep.h5')

#### Test

In [19]:
# perfs_test(model=interaction,test=Data[4],true=Data[5])

## Benchmark Test

In [20]:
def test_on_benchmark_pos_neg(model,w):
    print("Test on benchmark dataset")
    #model = load_model(filename)
    load = load_datatest_p_n(w)
    test = load[0]
    nb_pos = load[1]
    load=[]
    print("Data loaded")
    predictions = model.predict(test)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    true_y = np.zeros((len(predictions)))
    for i in range(nb_pos):
        true_y[i] = 1
    print(nb_pos,len(test[0]))
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])

## Run

In [27]:
test_on_benchmark_pos_neg(interaction,36)

Test on benchmark dataset
sequence :  uuucccugguguuggcgcaguauucgcgcaccccgg  de longueur :  36
sequence :  ugaaauucguuacaaggggaaauccguaaacccgcu  de longueur :  36
sequence :  auuucccugguguuggcgcaguauucgcgcaccccg  de longueur :  36
sequence :  ccaggagaaauagauggaaaccacucagaccagcac  de longueur :  36
sequence :  uuucccugguguuggcgcaguauucgcgcaccccgg  de longueur :  36
sequence :  ggauaugcaaucccagggagaaaauauguuuuugca  de longueur :  36
sequence :  cccugguguuggcgcaguauucgcgcaccccgguca  de longueur :  36
sequence :  acuguacauuuccaggguaacccggucaccguugcc  de longueur :  36
sequence :  augaaauuccucuuugacgggccaauagcgauauug  de longueur :  36
sequence :  cccgucaaagaguuauuucauaaaucaauaccgcaa  de longueur :  36
sequence :  uccucuuugacgggccaauagcgauauuggccauuu  de longueur :  36
sequence :  cgucaaagaggauuaacccaugcguacguuuagugg  de longueur :  36
sequence :  uccucuuugacgggccaauagcgauauuggccauuu  de longueur :  36
sequence :  cgucaaagaggauuaacuuaugcguacguuuagugg  de longueur :  36
sequence :  ccaggaacca

sequence :  gcaguggcaaucaucuccacccaggccuggucugca  de longueur :  36
sequence :  gccacugcuuuucuuugauguccccauuuuguggag  de longueur :  36
sequence :  gggauuaacaguggcaucgcauccgcagagaggcuu  de longueur :  36
sequence :  gccacugcuuuucuuugauguccccauuuuguggag  de longueur :  36
sequence :  gauugcaguggcacuggcugguuucgcuaccguagc  de longueur :  36
sequence :  gccacugcuuuucuuugauguccccauuuuguggag  de longueur :  36
sequence :  aagcaguggcauaaaaaagcaauaaaggcauauaac  de longueur :  36
sequence :  gccacugcuuuucuuugauguccccauuuuguggag  de longueur :  36
sequence :  aaguuaguggcaguggcagugacuucccuguuggca  de longueur :  36
sequence :  gccacugcuuuucuuugauguccccauuuuguggag  de longueur :  36
sequence :  ggcaguggcaggugucauauaaaaaaaccaaugagg  de longueur :  36
sequence :  gccacugcuuuucuuugauguccccauuuuguggag  de longueur :  36
sequence :  aaagaaaaguauuggcacuugucaucccggcucugc  de longueur :  36
sequence :  gccacugcuuuucuuugauguccccauuuuguggag  de longueur :  36
sequence :  agaaaaguucuggcacugcuugucccggcguuauua

sequence :  caauuggucugcgauucagaccacgguagcgagacu  de longueur :  36
sequence :  aaaaaguuugauugcagcgggaauacucacugcgcu  de longueur :  36
sequence :  gcgauucagaccaugguagcaaagcuaccuuuuuuc  de longueur :  36
sequence :  acuacgcaacguucaccggggcuauuccggcgucug  de longueur :  36
sequence :  guuguguuugcaauuggucugcgauucagaccacgg  de longueur :  36
sequence :  aauugauaucagggaucaagaugaaaaaaaaacuua  de longueur :  36
sequence :  ugcaauuggucugcgauucagaccacgguagcgaga  de longueur :  36
sequence :  aucagcaccaccaccauuaccaccaucaccauuacc  de longueur :  36
sequence :  guuugcaauuggucugcgauucagaccacgguagcg  de longueur :  36
sequence :  gucguuaccggcccggcgaauuauuuuucccaugca  de longueur :  36
sequence :  ugcaauuggucugcgauucagaccacgguagcgaga  de longueur :  36
sequence :  gaauaugaaauggcuacgcaacgagcaucagggcua  de longueur :  36
sequence :  agacaccauggacacaacguugagugaagcacccac  de longueur :  36
sequence :  auaaaucggaaucaaaaacuauguguggaauuguug  de longueur :  36
sequence :  agcgaugaaauuuuggccacuccgugaguggccuuu

[0.8778049  0.8321491  0.6258255  0.9871345  0.77271014 0.9562068
 0.84849566 0.7410587  0.6702922  0.6708024  0.88305664 0.71471596
 0.6780144  0.732153   0.2096966  0.85605264 0.8453204  0.74647355
 0.5915359  0.95711386 0.9460793  0.6197178  0.77499694 0.8497985
 0.8169804  0.8391932  0.96650976 0.99418926 0.95095426 0.8069529
 0.8480136  0.9869387  0.9032387  0.8473092  0.8499429  0.931696
 0.8143598  0.9904365  0.9209642  0.8684266  0.9700579  0.95073974
 0.88366455 0.9557024  0.9843939  0.99321014 0.5644113  0.6661938
 0.9231768  0.931098   0.8911667  0.917954   0.917954   0.97245854
 0.98738885 0.907776   0.82076216 0.5674138  0.57959896 0.48044744
 0.94902945 0.81111836 0.8338507  0.7718109  0.8595743  0.87150943
 0.94013214 0.7443339  0.9298921  0.8409724  0.965046   0.9832274
 0.8673767  0.9659232  0.8726914  0.9074255  0.84557474 0.87417287
 0.86234885 0.88325506 0.9227843  0.937407   0.9549142  0.8558551
 0.67351115 0.912219   0.8634704  0.7028596  0.95046735 0.9293514
 0.9

### Output

In [112]:
l1 = len(seq1)
l2 = len(seq2)

proba1 = np.array(RNA.pfl_fold_up(seq1,1,l1,l1))[:,1]
proba2 = np.array(RNA.pfl_fold_up(seq2,1,l2,l2))[:,1]

In [131]:
datatest = pd.read_csv("genomes/nar-03123-met-g-2015-File010.csv", sep = ",", header=None)
datatest = np.array(datatest)
i = 96
seq1 = traduire(datatest[i,23])
seq2 = traduire(datatest[i,24])

In [132]:
g = get_output(seq1,seq2,interaction, 36)

In [138]:
g96 = g

In [196]:
g96_s = get_output_s(seq1,seq2,10)

In [197]:
g96_p = g96*g96_s

In [198]:
g96_p[43,216]

87.78361129760742

In [199]:
len(np.argwhere(g96_p>87))

4218

In [ ]:
f = 1
conv = np.zeros((len(g1_fast)-8,g1_fast.shape[1]-8))
for k in range(len(g1_fast)-8):
    for p in range(g1_fast.shape[1]-8):
        conv[k,p] = np.sum(g1_fast[k:k+f,p:p+f])
conv[65:80,180:190]

In [ ]:
np.max(conv)

In [ ]:
len(np.argwhere(conv>0.998))

In [164]:
def get_output(seq1, seq2, model, window = 36):
    l1 = len(seq1)
    l2 = len(seq2)
    output = np.zeros((max(1,l1-window),max(1,l2-window)))
    proba1 = np.array(RNA.pfl_fold_up(seq1,1,l1,l1))[1:,1]
    proba2 = np.array(RNA.pfl_fold_up(seq2,1,l2,l2))[1:,1] 
    #proba1 = np.zeros(l1)
    #proba2 = np.zeros(l2)
    
    sequence1 = np.zeros((1,l1,4))
    sequence2 = np.zeros((1,l2,4))
    revseq1 = np.zeros((1,l1,4))
    revseq2 = np.zeros((1,l2,4))
    product = np.zeros((1,l1,l2,1))
    matrice3D = np.zeros((1,l1,l2,1))
    
    product[0] = get_sum_proba_fast(proba1,proba2)
    
    sequence1[0] = convert_sequence_fast(seq1)
    sequence2[0] = convert_sequence_fast(seq2)
    
    matrice3D[0] = convert_matrice3D_fast(sequence1[0],sequence2[0])
    
    for j in range(l1):
        revseq1[0][l1-j-1] = sequence1[0][j]
    for j in range(l2):
        revseq2[0][l2-j-1] = sequence2[0][j]
    
    
    for i in range(max(1,l1-window)):
        for j in range(max(1,l2-window)):
            test = []
            test.append(matrice3D[:,i:i+window,j:j+window])
            test.append(product[:,i:i+window,j:j+window])
            test.append(sequence1[:,i:i+window])
            test.append(revseq1[:,l1-i-window:l1-i])
            test.append(sequence2[:,j:j+window])
            test.append(revseq2[:,l2-j-window:l2-j])
            output[i,j] = (model.predict(test)[:,1])
            
    
    return output

In [195]:
def get_output_s(seq1, seq2, window = 36):
    l1 = len(seq1)
    l2 = len(seq2)
    output = np.zeros((max(1,l1-36),max(1,l2-36)))

    #proba1 = np.zeros(l1)
    #proba2 = np.zeros(l2)
    
    sequence1 = np.zeros((1,l1,4))
    sequence2 = np.zeros((1,l2,4))
    matrice3D = np.zeros((1,l1,l2,1))    
    
    sequence1[0] = convert_sequence_fast(seq1)
    sequence2[0] = convert_sequence_fast(seq2)
    
    matrice3D[0] = convert_matrice3D_fast(sequence1[0],sequence2[0])

    for i in range(max(1,l1-36)):
        for j in range(l2-36):
            r = 15*np.eye(window)+np.diag(np.ones(window-1),1)+np.diag(np.ones(window-1),-1)
            m = np.zeros((window,window))
            for q in range(window):
                m[:,q] = r[:,window-q-1]           
            output[i,j] = np.sum(matrice3D[0,i:i+window,j:j+window,0]*m)
#            print(output[i,j])
    
    return output

In [116]:
pos1 = 28
pos2 = 287
window = 36
test = transf(seq1[pos1:pos1+window],seq2[pos2:pos2+window],proba1[pos1:pos1+window],proba2[pos2:pos2+window])
output = interaction.predict(test)[:,1]
print(output)

[0.25701457]


In [27]:
def get_proba_interaction(seq1, seq2, model, pos1, pos2, window = 36):
    l1 = len(seq1)
    l2 = len(seq2)
    output = np.zeros((max(1,l1-window),max(1,l2-window)))
    proba1 = np.array(RNA.pfl_fold_up(seq1,1,l1,l1))[:,1]
    proba2 = np.array(RNA.pfl_fold_up(seq2,1,l2,l2))[:,1]    
    
    test = transf(seq1[pos1:pos1+window],seq2[pos2:pos2+window],proba1[pos1:pos1+window],proba2[pos2:pos2+window])
    output = (model.predict(test)[:,1])
    print(output)
    
    return output

In [28]:
def transf(seq1, seq2, proba1, proba2):
    #print(seq1)
    #print(seq2)
    
    sequence1 = np.zeros((1,36,4))
    sequence2 = np.zeros((1,36,4))
    revseq1 = np.zeros((1,36,4))
    revseq2 = np.zeros((1,36,4))
    product = np.zeros((1,36,36,1))
    matrice3D = np.zeros((1,36,36,1))
    
    product[0] = get_sum_proba(proba1,proba2)
    
    sequence1[0] = convert_sequence(seq1)
    sequence2[0] = convert_sequence(seq2)
    
    matrice3D[0] = convert_matrice3D(sequence1[0],sequence2[0])
    
    for j in range(36):
        revseq1[0][36-j-1] = sequence1[0][j]
        revseq2[0][36-j-1] = sequence2[0][j]
    
    test = []
    test.append(matrice3D)
    test.append(product)
    test.append(sequence1)
    test.append(revseq1)
    test.append(sequence2)
    test.append(revseq2)
    
    return test

# ARCHIVES

In [ ]:
def load_datatest1():
    datatest = pd.read_csv("genomes/nar-03123-met-g-2015-File010.csv", sep = ",", header=None)
    datatest = np.array(datatest)
    l = len(datatest)
    seq1 = np.zeros((200,36,4))
    seq2 = np.zeros((200,36,4))

    matrice3D = np.zeros((200,36,36,1))
    product = np.zeros((200,36,36,1))
    index = 0
    for i in range(1,l):
        print("i : ",i)
        begin1 = int(datatest[i,9])-1
        end1 = int(datatest[i,10])
        size1 = end1-begin1
        if(size1 <= 36):
    
            begin2 = int(datatest[i,11])-1
            end2 = int(datatest[i,12])
            size2 = end2-begin2
    
            if(size2 <= 36):
                proba1 = []
                proba2 = []
                print("taille 1 : ", size1)
                print("taille 2 : ", size2)
                print("index : ",index)
                sequence = traduire(datatest[i,23][begin1:end1])
                z1 = len(sequence)
                print("sequence : ",sequence, " de longueur : ", z1)
                proba1 = np.array(RNA.pfl_fold_up(sequence,1,z1,z1))[1:,1]
                seq1[index] = convert_sequence(sequence)
                
                sequence_tmp = traduire(datatest[i,24][begin2:end2])
                #sequence = reverse(sequence_tmp)
                sequence = sequence_tmp
                z2 = len(sequence)
                print("sequence : ",sequence, " de longueur : ", z2)
                proba2 = np.array(RNA.pfl_fold_up(sequence, 1, z2, z2))[1:,1]
                seq2[index] = convert_sequence(sequence)
                
                matrice3D[index] = convert_matrice3D(seq1[index],seq2[index])*sqrt(36*36/z1/z2)
                product[index] = get_sum_proba(proba1,proba2)*sqrt(36*36/z1/z2)
                
                
                index+=1
                               
    l = len(seq1)
    revseq1 = np.zeros((l,36,4))
    revseq2 = np.zeros((l,36,4))
    
    for i in range(l):
        for j in range(36):
            revseq1[i][36-j-1] = seq1[i][j]
            revseq2[i][36-j-1] = seq2[i][j]
    
    test = []
    test.append(matrice3D[:index])
    test.append(product[:index])
    test.append(seq1[:index])
    test.append(revseq1[:index])
    test.append(seq2[:index])
    test.append(revseq2[:index])
    
    return test

def load_datatest2():
    datatest = pd.read_csv("genomes/nar-03123-met-g-2015-File010.csv", sep = ",", header=None)
    datatest = np.array(datatest)
    l = len(datatest)
    seq1 = np.zeros((200,36,4))
    seq2 = np.zeros((200,36,4))
    #seq3 = np.zeros((200,36,4))
    #seq4 = np.zeros((200,36,4))
    matrice3D = np.zeros((200,36,36,1))
    product = np.zeros((200,36,36,1))
    index = 0
    for i in range(1,l):
        print("i : ",i)
        begin1 = int(datatest[i,9])-1
        end1 = int(datatest[i,10])
        #milieu1 = int((begin1+end1)/2)
        size1 = end1-begin1
        if(size1 <= 36):
    
            begin2 = int(datatest[i,11])-1
            end2 = int(datatest[i,12])
            size2 = end2-begin2
    
            if(size2 <= 36):
                proba1 = []
                proba2 = []
                print("taille 1 : ", size1)
                print("taille 2 : ", size2)
                print("index : ",index)
                
                sequence = traduire(datatest[i,23][begin1:begin1+36])
                
                z1 = len(sequence)
                print("sequence : ",sequence, " de longueur : ", z1)
                proba1 = np.array(RNA.pfl_fold_up(sequence,1,z1,z1))[1:,1]
                seq1[index] = convert_sequence(sequence)
                
                sequence_tmp = traduire(datatest[i,24][begin2:begin2+36])
                sequence = sequence_tmp
                z2 = len(sequence)
                print("sequence : ",sequence, " de longueur : ", z2)
                proba2 = np.array(RNA.pfl_fold_up(sequence, 1, z2, z2))[1:,1]
                seq2[index] = convert_sequence(sequence)
                
                matrice3D[index] = convert_matrice3D(seq1[index],seq2[index])
                product[index] = get_sum_proba(proba1,proba2)
                
                
                index+=1
                               
    l = len(seq1)
    revseq1 = np.zeros((l,36,4))
    revseq2 = np.zeros((l,36,4))
    
    for i in range(l):
        for j in range(36):
            revseq1[i][36-j-1] = seq1[i][j]
            revseq2[i][36-j-1] = seq2[i][j]
    
    test = []
    test.append(matrice3D[:index])
    test.append(product[:index])
    test.append(seq1[:index])
    test.append(revseq1[:index])
    test.append(seq2[:index])
    test.append(revseq2[:index])
    
    return test

In [ ]:
def test_on_benchmark1(model):
    print("Test on benchmark dataset")
    #model = load_model(filename)
    test = load_datatest1()
    print("Data loaded")
    predictions = model.predict(test)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    true_y = np.ones((len(predictions)))
    
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])

def test_on_benchmark2(model):
    print("Test on benchmark dataset")
    #model = load_model(filename)
    test = load_datatest2()
    print("Data loaded")
    predictions = model.predict(test)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    true_y = np.ones((len(predictions)))
    
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])

In [ ]:
def load_dataset():
    # load your data using this function
    neg1 = []

    neg1 = pd.read_csv("genomes/negatifs_m-m-str.csv", sep = "\t",header=None)
    neg1 = np.array(neg1)
    print(len(neg1))
    for i in range(len(neg1)):
        for j in range(2):
            if len(neg1[i,j]) <= 15:
                print(i,"erreur")
            

    neg2 = pd.read_csv("genomes/negatifs_divers-str.csv", sep = "\t",header=None)
    neg2 = np.array(neg2)
    print(len(neg2))
    for i in range(len(neg2)):
        for j in range(2):
            if len(neg2[i,j]) <= 15:
                print(i,"erreur")
    
    neg3 = pd.read_csv("genomes/negatifs_mouse_divers-str.csv", sep = "\t",header=None)
    neg3 = np.array(neg3)
    print(len(neg3))
    for i in range(len(neg3)):
        for j in range(2):
            if len(neg3[i,j]) <= 15:
                print(i,"erreur")

    pos1 = pd.read_csv("genomes/positifs_m-m-str.csv", sep = "\t",header=None)  
    pos1 = np.array(pos1)
    print(len(pos1))
    for i in range(len(pos1)):
        for j in range(2):
            if len(pos1[i,j]) <= 15:
                print(i,"erreur")
    l = len(pos1)
    
    pos2 = pd.read_csv("genomes/positifs_divers-str.csv", sep = "\t",header=None)  
    pos2 = np.array(pos2)
    print(len(pos2))
    for i in range(len(pos2)):
        for j in range(2):
            if len(pos2[i,j]) <= 15:
                print(i,"erreur")
    l=l+len(pos2)

    pos3 = pd.read_csv("genomes/positifs_mouse_divers-str.csv", sep = "\t",header=None)  
    pos3 = np.array(pos3)
    print(len(pos3))
    for i in range(len(pos3)):
        for j in range(2):
            if len(pos3[i,j]) <= 15:
                print(i,"erreur")
    l=l+len(pos3)

    bdd = np.concatenate((pos1,pos2,pos3,neg1,neg2,neg3))

    pos1 = []
    neg1 = []
    pos2 = []
    neg2 = []
    pos3 = []
    neg3 = []

    labels = np.zeros((len(bdd),1))

    bdd = np.concatenate((bdd,labels),axis=1)
    for i in range(l):
        bdd[i,4]=1
    labels=[]
    
    # shuffle total

    indices = np.arange(len(bdd))
    shuffle(indices)
    bdd = bdd[indices]
    indices=[]

    l = len(bdd)
    matrice = np.zeros((l,36,36,1))
    matrice1 = np.zeros((l,36,36,1))
    matrice2 = np.zeros((l,36,4))
    matrice3 = np.zeros((l,36,4))
    matrice4 = np.zeros((l,36,4))
    matrice5 = np.zeros((l,36,4))

    for i in range(l):
        seq1 = bdd[i,0]
        seq2 = bdd[i,1]
        prob1 = np.array(clean(bdd[i,2]))
        prob2 = np.array(clean(bdd[i,3]))
        for j in range(len(seq1)):
            for k in range(len(seq2)):
                if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                    matrice[i,j,k,0] = 1
                elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                matrice1[i,j,k,0] = prob1[j]+prob2[k]
        for j in range(len(seq1)):
            if seq1[j]=='a':
                matrice2[i,j,0] = 1
            elif seq1[j]=='u':
                matrice2[i,j,1] = 1
            elif seq1[j]=='g':
                matrice2[i,j,2] = 1
            elif seq1[j]=='c':
                matrice2[i,j,3] = 1
        for j in range(len(seq2)):
            if seq2[j]=='a':
                matrice3[i,j,0] = 1
            elif seq2[j]=='u':
                matrice3[i,j,1] = 1
            elif seq2[j]=='g':
                matrice3[i,j,2] = 1
            elif seq2[j]=='c':
                matrice3[i,j,3] = 1
                
    for i in range(36):
        matrice4[:,36-i-1,:] = matrice2[:,i,:]
    for i in range(36):
        matrice5[:,36-i-1,:] = matrice3[:,i,:]
                
    labels = bdd[:,4]
    bdd = []
    
    return matrice, matrice1, matrice2, matrice3, matrice4, matrice5, labels

def load_data(nb_train = 500,nb_val = 600,nb_test = 2500):
    data = load_dataset()
    matrice = data[0]
    matrice1 = data[1]
    matrice2 = data[2]
    matrice3 = data[3]
    matrice4 = data[4]
    matrice5 = data[5]
    labels = data[6]
    
    training = []
    training.append(matrice[:nb_train])
    training.append(matrice1[:nb_train])
    training.append(matrice2[:nb_train])
    training.append(matrice4[:nb_train])
    training.append(matrice3[:nb_train])
    training.append(matrice5[:nb_train])

    validation = []
    validation.append(matrice[nb_train:nb_train+nb_val])
    validation.append(matrice1[nb_train:nb_train+nb_val])
    validation.append(matrice2[nb_train:nb_train+nb_val])
    validation.append(matrice4[nb_train:nb_train+nb_val])
    validation.append(matrice3[nb_train:nb_train+nb_val])
    validation.append(matrice5[nb_train:nb_train+nb_val])


    testing = []
    testing.append(matrice[-nb_test:])
    testing.append(matrice1[-nb_test:])
    testing.append(matrice2[-nb_test:])
    testing.append(matrice4[-nb_test:])
    testing.append(matrice3[-nb_test:])
    testing.append(matrice5[-nb_test:])
    
    y = labels[:nb_train]
    y = keras.utils.np_utils.to_categorical(y,2)
    val_y = labels[nb_train:nb_train+nb_val]
    val_y = keras.utils.np_utils.to_categorical(val_y,2)
    true_y = labels[-nb_test:]
    argtest=[]
    return training, y, validation, val_y, testing, true_y

In [ ]:
# paramètres 
#true_y = Data[5]
#training = Data[0]
#y = Data[1]
batch_size=64
epochs=25
verbose1 = 1
verbose2 = 1
#validation = Data[2]
#val_y = Data[3]
#testing = Data[4]
matrixsize11 = 6
nbfilter11 = 24
matrixsize12 = 7
nbfilter12 = 4
matrixsize21 = 6
nbfilter21 = 24
matrixsize22 = 7
nbfilter22 = 4
nbfilter1 = 96
kernelsize = 7
nbfilters2 = 96
kernel_size2 = 7
Dense1 = 128
Dense2 = 512
Dense3 = 512
Dense4 = 128
Dense5 = 64

from keras.layers import merge
from keras import Model
k = matrixsize11
# init_weights
I = np.eye(k)
M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
I2 = np.zeros((k,k))
M2 = np.zeros((k,k))
for j in range(k):
    I2[:,j] = I[:,k-j-1]
    M2[:,j] = M[:,k-j-1]        
W = np.zeros((k,k,1,nbfilter11))
W[:,:,0,0] = I
W[:,:,0,1] = I2
W[:,:,0,2] = M
W[:,:,0,3] = M2
for j in range(4,nbfilter11):
    W[:,:,0,j] = np.random.randn(k,k)*0.2


k2 = matrixsize12
I = np.eye(k2)
M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
I2=np.zeros((k2,k2))
M2=np.zeros((k2,k2))
for j in range(k2):
    I2[:,j] = I[:,k2-j-1]
    M2[:,j] = M[:,k2-j-1]   

Z = np.zeros((k2,k2,nbfilter11,nbfilter12))

for u in range(nbfilter12):
    Z[:,:,u,0] = I
    Z[:,:,u,1] = I2
    Z[:,:,u,2] = M
    Z[:,:,u,3] = M2
    for p in range(4,nbfilter12):
        Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

c2d1_input = keras.Input(shape=(36,36,1))
cnn2d1 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d1_input)
cnn2d1 = AveragePooling2D(pool_size=(3,3))(cnn2d1)
cnn2d1 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d1)
cnn2d1 = Dropout(0.1)(cnn2d1)
cnn2d1 = Flatten()(cnn2d1)
#cnn2d1 = get_cnn_network(matrixsize11, nbfilter11, matrixsize12, nbfilter12)(c2d1_input)
#training_net1.append(cnn2d1)
c2d2_input = keras.Input(shape=(36,36,1))
cnn2d2 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d1_input)
cnn2d2 = AveragePooling2D(pool_size=(3,3))(cnn2d2)
cnn2d2 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d2)
cnn2d2 = Dropout(0.1)(cnn2d2)
cnn2d2 = Flatten()(cnn2d2)
#cnn2d2 = get_cnn_network(matrixsize21, nbfilter21, matrixsize22, nbfilter22)(c2d2_input)
#training_net1.append(cnn2d2)
c1d1_input = keras.Input(shape=(36,4))
cnn1d1 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d1_input)
cnn1d1 = MaxPooling1D(pool_size=8)(cnn1d1)
cnn1d1 = Dropout(0.2)(cnn1d1)
cnn1d1 = Flatten()(cnn1d1)
#cnn1d1 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d1_input)
c1d2_input = keras.Input(shape=(36,4))
cnn1d2 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d2_input)
cnn1d2 = MaxPooling1D(pool_size=8)(cnn1d2)
cnn1d2 = Dropout(0.2)(cnn1d2)
cnn1d2 = Flatten()(cnn1d2)
#cnn1d2 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d2_input)
c1d3_input = keras.Input(shape=(36,4))
cnn1d3 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d3_input)
cnn1d3 = MaxPooling1D(pool_size=8)(cnn1d3)
cnn1d3 = Dropout(0.2)(cnn1d3)
cnn1d3 = Flatten()(cnn1d3)
#cnn1d3 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d3_input)
c1d4_input = keras.Input(shape=(36,4))
cnn1d4 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d4_input)
cnn1d4 = MaxPooling1D(pool_size=8)(cnn1d4)
cnn1d4 = Dropout(0.2)(cnn1d4)
cnn1d4 = Flatten()(cnn1d4)
#cnn1d4 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d4_input)
#training_net21.append(cnn1d1)
#training_net21.append(cnn1d2)
#training_net22.append(cnn1d3)
#training_net22.append(cnn1d4)

#model21 = Sequential()
model21 = keras.layers.concatenate([cnn1d1,cnn1d2])
model21 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model21)
model21 = BatchNormalization()(model21)
model21 = Activation('relu')(model21)
model21 = Dropout(0.2)(model21)
#model22 = Sequential()
model22 = keras.layers.concatenate([cnn1d3,cnn1d4])
model22 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model22)
model22 = BatchNormalization()(model22)
model22 = Activation('relu')(model22)
model22 = Dropout(0.2)(model22)
#training_net2.append(model21)
#training_net2.append(model22)
#model2 = Sequential()
model2 = keras.layers.concatenate([model21,model22])
model2 = Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model2)
model2 = BatchNormalization()(model2)
model2 = Activation('relu')(model2)
model2 = Dropout(0.2)(model2)
#model2.add(Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
#model2.add(BatchNormalization())
#model2.add(Activation('relu'))
#model2.add(Dropout(0.2))

#model1 = Sequential()
model1 = keras.layers.concatenate([cnn2d1,cnn2d2])
model1 = Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model1)
model1 = Dropout(0.1)(model1)
model1 = BatchNormalization()(model1)
model1 = Activation('relu')(model1)

#training_net.append(model1)
#training_net.append(model2)
#model = Sequential()
model = keras.layers.concatenate([model1,model2])
model = Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.1)(model)
model = Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
#model.add(Dropout(0.1))
if Dense5>0:
    model = Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
model = Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
interaction_output = Activation('softmax')(model)
#model.add(Activation('softmax'))
interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
interaction.compile(loss='categorical_crossentropy', optimizer='rmsprop')

earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose = verbose1)
print('model training')
#model.fit(training, y, batch_size, epochs, verbose = verbose2, validation_data=(validation, val_y), callbacks=[earlystopper])
#interaction.fit(training, y, batch_size, epochs, verbose = verbose2, validation_data=(validation, val_y), callbacks=[earlystopper])
# test   

print('predicting')

In [ ]:
interaction.get_weights()